# 二次関数と接線の動画化
二次関数と接線3つをIPythonとMP4とGIFいずれかに出力するコードです。  
動作確認済み環境は以下の通り。

項目|バージョン
---|---
OS|Mac OSX 10.10.5(Yosemite)
Python|3.5.1
matplotlib|1.5.1
numpy|1.11.0
seaborn|0.7.0
ipython|4.1.2

## 必要なライブラリのインポート
[matplotlib.animation](http://matplotlib.org/api/animation_api.html)はアニメーション化に必要なパッケージです。  
[IPython.display.HTML](https://ipython.org/ipython-doc/3/api/generated/IPython.display.html#IPython.display.HTML)はアニメーションのIPython内での表示に必要なクラスです。

In [12]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation
from IPython.display import HTML,display

## プロット用のオブジェクトの作成
動画は基本的にパラパラ漫画なので、動かすグラフは前もってインスタンスを生成し、  
更新用のメソッド内に渡してあげる必要があります。

In [13]:
# プロット用のオブジェクトを作成
fig, ax = plt.subplots()
plt.close()
# X軸を-6〜6までに設定
ax.set_xlim((-6, 6))
# Y軸を-2〜20までに設定
ax.set_ylim((-2, 20))

# 接線描写用のオブジェクトを作成
line, = ax.plot([], [], lw=1)
# 接点描写用のオブジェクトを作成
scat = ax.scatter([],[], color='gray',marker='o',s=40)


## 背景の描画
二次曲線自体は動画の中では動かすものではないので、  
ここで先に描画して、背景のように扱います。

In [14]:
# 二次曲線の描画
quadratic_curve_x = np.arange(-16,16,0.01) 
quadratic_curve_y = quadratic_curve_x**2.0
ax.plot(quadratic_curve_x, quadratic_curve_y, linewidth=3)

## 更新用メソッドの定義

In [15]:
# コールバックメソッドの定義
def animate(i,scat,line):
    # フレームからxの位置を算出（定義域としては、FRAMESの値が100であれば
    # iは全フレームを通して-5.0〜5.0を推移する）
    x = (i - (FRAMES / 2)) / 10

    # あるフレームでの接線の描画
    # 接線の式は「y = 2ax -a^2」で表される
    # この時、aが二次曲線との接点のxの値となる
    # aの値を原点としてプラスマイナスそれぞれの方向に16ずつ伸ばした線分を描画する
    line_x = np.arange(-16,16,0.01) 
    line_y = 2.0*x*line_x - x**2.0
    line.set_data(line_x, line_y)
    
    # あるフレームでの接点の描画
    # フレーム毎のx値におけるy値は基本的に二次曲線で算出するy値と同じになる
    scat_x = x 
    scat_y = scat_x**2.0
    scat.set_offsets((scat_x,scat_y))

## アニメーションの生成
ここでFuncAnimationメソッドを利用してアニメーションを生成します。
メソッドの引数は以下の通り。

引数|意味
---|---
fig|描画領域
func|フレーム毎に更新する描写を定義したコールバック関数
fargs|更新するグラフの関数
frames|描画全体のフレーム数
interval|何ミリ秒毎に再描画するか（アニメーションのコマを進めるか）

In [16]:
# 動画全体のフレームを設定する
FRAMES = 100

# アニメーションを作成する関数FuncAnimationを定義する。
anim = animation.FuncAnimation(fig = fig, func = animate, fargs = (scat,line), frames = FRAMES, interval=30)

## アニメーションの出力
アニメーションの出力先によって方法が異なります。以下の通り

出力先|方法
---|---
IPython内に表示する|HTMLクラスを利用する
MP4やGIF等ファイルに出力する|Writerクラスを利用する

また、Writerクラスをインスタンス化する際の引数は以下の通りになります。

引数|意味
---|---
fps|フレームレート（ほとんどの動画サイトは30fps以下であればアップロードできる。<br>また、そんなにないと思うけれどDVD化等に動画を出力する際は<br>29.97fpsにする必要がある。そのため、ここでは29.97に設定）
metadata|動画作成者等のメタデータを設定
bitrate|動画のビットレート（画質）

In [17]:
export_to = "IPython"
#export_to = "MP4"
#export_to = "GIF"

title = "x**2"
if export_to == "IPython":
    # HTML化することで描画
    display(HTML(anim.to_html5_video()))
elif export_to == "MP4":
    # FFmpegというUNIX系の動画変換ソフトを利用する
    Writer = animation.writers['ffmpeg']
    # エクスポートされるファイルの情報を定義してインスタンス化する
    writer = Writer(fps=29.97, metadata=dict(artist='Me'), bitrate=1800)
    # 実際の動画のファイルエスクスポート
    anim.save(title + '.mp4', writer=writer)
elif export_to == "GIF":
    # FFmpegというUNIX系の動画変換ソフトを利用する
    Writer = animation.writers['imagemagick']
    # エクスポートされるファイルの情報を定義してインスタンス化する
    writer = Writer(fps=29.97, metadata=dict(artist='Me'), bitrate=1800)
    # 実際の動画のファイルエスクスポート
    anim.save(title + '.gif', writer=writer)
